# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [18]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', 
                 max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        self.get_feature_ids = self.__get_feature_ids_sqrt
        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return ( l_s - (np.sum(l_c * l_c / l_s, axis=1, keepdims=True))
               + r_s - (np.sum(r_c * r_c / r_s, axis=1, keepdims=True)))  # Ваш код
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(np.sum((l_c * np.log2(((l_c / l_s).clip(min=1e-3, max=1-1e-3))) / l_s)
                      + (r_c * np.log2(((r_c / r_s).clip(min=1e-3, max=1-1e-3))) / r_s), axis=1, keepdims=True)) # Ваш код 

    def __misclass(self, l_c, l_s, r_c, r_s):
        return (l_s + r_s) - (np.max(l_c + r_c, axis=1, keepdims=True))  # Ваш код

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[0:int(np.sqrt(n_feature))] # Ваш код
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[0:int(np.log2(n_feature))] # Ваш код

    def __get_feature_ids_N(self, n_feature):
        feature_ids = range(n_feature)
        return feature_ids[0:n_feature]               # Ваш код
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        ## Упорядочиваем первый набор по возрастанию, а второй - согласованно с ним.
        ## Определяем количество лейблов для кластеров (количество возможных классов)  
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        ## Выделяем область для анализа: крайние правые и крайние левые элементы 
        ## в количестве (min_samples_split) считаем заведомо принадлежащими разным узлам
        ## Замечание: если элементов меньше (2 * min_samples_split) + 2, дальнейший анализ невозможен
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
                 
        ## Определяем индексы внутри области, на которых происходит смена кластера ---
        ## это кандидаты для следующего разветвления
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        ## Если смен не было - всё принадлежит одному кластеру, 
        ## а значит и одному узлу, работа завершена
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        ## Определяем количество элементов в каждом из полученных кусочков,
        ## чтобы затем оценить хорошесть каждого из вариантов распила
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        ## Используем унитарный код для границ групп и номеров соответствующих кластеров         
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        ## Построим матрицу изменения распределения классов по узлам при изменении значения threshold:
        ## Matr[группа G (определяется по правой границе)][номер N класса] = 
        ##         количество элементов класса N из группы G, перешедших из левого узла в правый узел дерева
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        #class_increments[0] = class_increments[0] + np.bincount(y[:self.min_samples_split], minlength=class_number)
        class_increments[0] = class_increments[0] + np.bincount(y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        ## Выделяем матричные параметры для оценки вариантов распила:
        ## количество элементов, попавших в каждый из узлов (левый и правый) 
        ## Matr[группа G (определяется по правой границе)][номер N класса] = 
        ##         количество элементов класса N из группы G со значением x < sorted_x[G]
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        ## И количество элементов в каждой группе
        ## при реализации каждого из возможных распилов.
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        ## Оценка вариантов распила и выбор наилучшего (минимального)
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        # Что делает этот блок кода?
        ## Номер элемента, после которого будет совершен распил
        left_el_id = l_sizes[idx][0]
        threshold = (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0
        ## Оценка распила и текущее значение threshold
        return gs[idx], threshold

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        
        # Количество уникальных меток классов и вероятности попадания в каждый отдельный класс
        classes_counts = np.bincount(y, minlength=self.num_class)
        classes_probs = classes_counts / np.sum(classes_counts).astype('float')
        
        # Если достигли максимальной глубины - остаток разбора отправляется в один лист
        if self.max_depth is not None and depth == self.max_depth :
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(classes_counts), classes_probs)
            return
        # Если отстаток разбора меньше, чем допустимое минимальное количество элементов для распила - в один лист
        if self.min_samples_split is not None and x.shape[0] < self.min_samples_split:
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(classes_counts), classes_probs)
            return
        
        # Получим оценки распилов и пороговые значения
        gs = np.empty(x.shape[1])
        thresholds = np.empty(x.shape[1])
        
        # для этого выделим первый из классов
        first_label = np.min(y)
        y -= first_label
        
        # найдём оценки и пороговые значения для распилов 
        for index in self.get_feature_ids(x.shape[1]):
            gs[index], thresholds[index] = self.__find_threshold(x[:, index], y)
            
        y += first_label
        first_group_index = np.argmin(gs)
                
        # Не удалось найти хороший распил
        if np.isnan(thresholds[first_group_index]):
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(classes_counts), classes_probs)
            return
        
        # Индексы элементов левой и правой групп
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, first_group_index, thresholds[first_group_index])
        
        # Если какая-то из групп оказалась пуста
        if y_l.shape[0] == 0 or y_r.shape[0] == 0:
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(classes_counts), classes_probs)
            return
        
        # Создание узловой вершины
        self.tree[node_id] = (self.NON_LEAF_TYPE, first_group_index, thresholds[first_group_index])
        # Рекурсия пострения для каждого из потомков
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
        #pass
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [19]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_depth=5, criterion='gini')
clf = DecisionTreeClassifier(min_samples_split=2, min_samples_leaf=1, max_depth=5, criterion='gini')

## Проверка скорости работы

In [21]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.488510847092
0.633349895477


## Проверка качества работы

In [22]:
gkf = KFold(n_splits=5, shuffle=True)

In [23]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.935145921676
0.936018957346
0.932027937141
0.935561652948
0.933064482601


In [24]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.932651534048
0.933524569718
0.932152656523
0.933482996591
0.935974722488
